## Level 2

In [ ]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BidData-HW-Level2").getOrCreate()

In [ ]:
from pyspark import SparkFiles
# Load in employee.csv from S3 into a DataFrame
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Toys_v1_00.tsv.gz"
spark.sparkContext.addFile(url)

In [ ]:
toys_df=spark.read.csv(SparkFiles.get("amazon_reviews_us_Toys_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True, timestampFormat="mm/dd/yy")
toys_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   18778586| RDIJS7QYB6XNR|B00EDBY7X8|     122952789|Monopoly Junior B...|            Toys|          5|            0|          0|   N|                Y|          Five Stars|        Excellent!!!| 2015-08-31|
|         US|   24769659|R36ED1U38IELG8|B00D7JFOPC|     952062646|56 Pieces of Wood...|            Toys|          5|    

In [ ]:
# Filter by votes
filtered_df = toys_df.select(["star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
filtered_df.show()

+-----------+-------------+-----------+----+-----------------+
|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+-----------+-------------+-----------+----+-----------------+
|          5|            0|          0|   N|                Y|
|          5|            0|          0|   N|                Y|
|          2|            1|          1|   N|                Y|
|          5|            0|          0|   N|                Y|
|          1|            1|          1|   N|                Y|
|          5|            0|          0|   N|                Y|
|          3|            2|          2|   N|                Y|
|          5|            0|          0|   N|                Y|
|          5|            0|          0|   N|                Y|
|          1|            0|          1|   N|                Y|
|          5|            0|          0|   N|                Y|
|          5|            0|          0|   N|                Y|
|          5|            0|          1|   N|           

In [ ]:
total_votes = filtered_df.filter(filtered_df["total_votes"] >= 20)
total_votes.show()

+-----------+-------------+-----------+----+-----------------+
|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+-----------+-------------+-----------+----+-----------------+
|          5|           23|         27|   N|                Y|
|          1|           21|         21|   N|                Y|
|          5|           19|         20|   Y|                N|
|          4|           59|         81|   N|                N|
|          5|           22|         23|   Y|                N|
|          3|            9|         21|   N|                Y|
|          3|           33|         33|   Y|                N|
|          4|           50|         50|   N|                N|
|          3|           84|         84|   N|                Y|
|          5|           20|         20|   N|                Y|
|          5|           22|         22|   N|                N|
|          1|           28|         36|   N|                Y|
|          5|           42|         43|   N|           

In [ ]:
help_votes = total_votes.filter(total_votes["helpful_votes"]/total_votes["total_votes"] >= .5)
help_votes.show()

+-----------+-------------+-----------+----+-----------------+
|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+-----------+-------------+-----------+----+-----------------+
|          5|           23|         27|   N|                Y|
|          1|           21|         21|   N|                Y|
|          5|           19|         20|   Y|                N|
|          4|           59|         81|   N|                N|
|          5|           22|         23|   Y|                N|
|          3|           33|         33|   Y|                N|
|          4|           50|         50|   N|                N|
|          3|           84|         84|   N|                Y|
|          5|           20|         20|   N|                Y|
|          5|           22|         22|   N|                N|
|          1|           28|         36|   N|                Y|
|          5|           42|         43|   N|                N|
|          4|           32|         34|   N|           

In [ ]:
paid_df = help_votes.filter(help_votes["vine"] == "Y")
paid_df.describe().show(10)

+-------+------------------+------------------+-----------------+----+-----------------+
|summary|       star_rating|     helpful_votes|      total_votes|vine|verified_purchase|
+-------+------------------+------------------+-----------------+----+-----------------+
|  count|              1266|              1266|             1266|1266|             1266|
|   mean|3.8894154818325433|53.947077409162716|58.99605055292259|null|             null|
| stddev| 1.040111793724077| 66.31016742328943|  69.742173291185|null|             null|
|    min|                 1|                12|               20|   Y|                N|
|    max|                 5|               788|              850|   Y|                Y|
+-------+------------------+------------------+-----------------+----+-----------------+



In [ ]:
unpaid_df = help_votes.filter(help_votes["vine"] == "N")
unpaid_df.describe().show(10)

+-------+------------------+------------------+------------------+-----+-----------------+
|summary|       star_rating|     helpful_votes|       total_votes| vine|verified_purchase|
+-------+------------------+------------------+------------------+-----+-----------------+
|  count|             62028|             62028|             62028|62028|            62028|
|   mean| 3.698426517056813| 49.12094537950603| 53.79865544592764| null|             null|
| stddev|1.5470270527750734|126.32476362651767|131.01895930767017| null|             null|
|    min|                 1|                10|                20|    N|                N|
|    max|                 5|             12188|             12380|    N|                Y|
+-------+------------------+------------------+------------------+-----+-----------------+



In [ ]:
paid_five = paid_df[paid_df["star_rating"] == 5].count()
paid_num= paid_df.count()
percentage_five= round(float(paid_five) / float(paid_num),4)
percentage_five

0.3412

In [ ]:
print(f"Number of Paid Review {paid_num}")
print(f"Number of Paid 5 Star Review {paid_five}")
print(f"Percent of Paid Review {percentage_five * 100}%")

Number of Paid Review 1266
Number of Paid 5 Star Review 432
Percent of Paid Review 34.12%


In [ ]:
unpaid_five = unpaid_df[paid_df["star_rating"] == 5].count()
unpaid_num= unpaid_df.count()
percentage_five= round(float(unpaid_five) / float(unpaid_num),4)
percentage_five

0.4834

In [ ]:
print(f"Number of UnPaid Review {unpaid_num}")
print(f"Number of UnPaid 5 Star Review {unpaid_five}")
print(f"Percent of UnPaid Review {percentage_five * 100}%")

Number of UnPaid Review 62028
Number of UnPaid 5 Star Review 29982
Percent of UnPaid Review 48.339999999999996%


### Analysis

Based on the numbers above the percentage of five star reviews given are not impacted by the "paid" review status. 